In [3]:
!pip install -q pandas
!pip install dask
!pip install pyarrow
!pip install fastparquet phonetics fuzzywuzzy jellyfish -q
!pip install seaborn -q 
!pip install matplotlib -q 

import pandas as pd
!pip install recordlinkage
import recordlinkage as rl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dask.dataframe as dd
import dask
from recordlinkage.preprocessing import clean, phonetic

# get cpu number
import multiprocessing
multiprocessing.cpu_count()

# get RAM
!cat /proc/meminfo

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
MemTotal:       258379260 kB
MemFree:        255093988 kB
MemAvailable:   255342780 kB
Buffers:            2704 kB
Cached:          2114680 kB
SwapCached:            0 kB
Active:           555096 kB
Inactive:        2294464 kB
Active(anon):        536 kB
Inactive(anon):   732440 kB
Active(file):     554560 kB
Inactive(file):  1562024 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:                 8 kB
Writeback:             0 kB
AnonPages:        732300 kB
Mapped:           305612 kB
Shmem:               800 kB
KReclaimable:     111052 kB
Slab:             226164 kB
SReclaimable:     111052 kB
SUnreclaim:       115112 kB
KernelStack:        9872 kB
PageTables:        12488 kB
NFS_Uns

In [4]:
from dask.distributed import LocalCluster
cluster = LocalCluster(n_workers=8, threads_per_worker=4)

## Read

In [6]:
# Patent from CSV
US_patent_csv = pd.read_csv("../ori_data/US_patent/PatentsDataUS_Clean.csv")

/tmp/ipykernel_13865/1870751802.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  US_patent_csv = pd.read_csv("../ori_data/US_patent/PatentsDataUS_Clean.csv")


In [18]:
US_patent_csv

,publicationnumber,X,Publication.date,Title,Description,inventorsnames1,inventorsnames2,inventorsnames3,inventorsnames4,inventorsnames5,inventorsnames6,year
0,US100000A,1,22/02/1870,NaN,"<description lang=""en"" load-source=""lhip"" data...",NaN,NaN,NaN,NaN,NaN,NaN,1870
1,US100001A,2,22/02/1870,NaN,"<description lang=""en"" load-source=""lhip"" data...",NaN,NaN,NaN,NaN,NaN,NaN,1870
2,US100002A,3,22/02/1870,NaN,"<description lang=""en"" load-source=""lhip"" data...",NaN,NaN,NaN,NaN,NaN,NaN,1870
3,US100003A,4,22/02/1870,NaN,"<description lang=""en"" load-source=""lhip"" data...",NaN,NaN,NaN,NaN,NaN,NaN,1870
4,US100004A,5,22/02/1870,NaN,"<description lang=""en"" load-source=""lhip"" data...",NaN,NaN,NaN,NaN,NaN,NaN,1870
...,...,...,...,...,...,...,...,...,...,...,...,...
309030,USRE8396E,18494,03/09/1878,NaN,"<description lang=""en"" load-source=""lhip"" data...",NaN,NaN,NaN,NaN,NaN,NaN,1878
309031,USRE8666E,18495,08/04/1879,NaN,"<description lang=""en"" load-source=""lhip"" data...",NaN,NaN,NaN,NaN,NaN,NaN,1879
309032,USRE9467E,18497,16/11/1880,NaN,"<description lang=""en"" load-source=""lhip"" data...",NaN,NaN,NaN,NaN,NaN,NaN,1880
309033,USRE9656E,18498,12/04/1881,NaN,"<description lang=""en"" load-source=""lhip"" data...",NaN,NaN,NaN,NaN,NaN,NaN,1881


In [8]:
US_patent_csv.year.value_counts()

year
1890    22409
1885    20618
1889    20212
1886    19296
1883    18719
1887    17962
1888    17176
1884    16807
1882    16133
1881    14094
1876    13225
1875    12183
1877    12017
1880    11829
1878    11462
1874    11269
1879    11170
1870    10910
1872    10702
1873    10416
1871    10415
1891        3
1869        3
1895        2
1892        1
1894        1
1893        1
Name: count, dtype: int64

In [9]:
%%time
# Census from dta
US_Census1880_dta = pd.read_stata("../ori_data/US_patent/US_Census1880.dta")


In [12]:
US_Census1880_dta

,namefrst,namelast,sursim,age,sex,occhisco,ind95us,enumdist,countyus,metrous,us1880a_region,us1880a_stateicp,us1880a_city,us1880a_stcounty,us1880a_county,us1880a_street
0,JAMES,GREEN,1.0,34.0,1.0,75000.0,439.0,1,10,0,32.0,41.0,0.0,10010.0,10.0,
1,MARY,GREEN,1.0,27.0,2.0,99999.0,0.0,1,10,0,32.0,41.0,0.0,10010.0,10.0,
2,ELLA,GREEN,1.0,11.0,2.0,99999.0,0.0,1,10,0,32.0,41.0,0.0,10010.0,10.0,
3,JESSE,GREEN,1.0,9.0,1.0,99999.0,0.0,1,10,0,32.0,41.0,0.0,10010.0,10.0,
4,FRANK,GREEN,1.0,7.0,1.0,99999.0,0.0,1,10,0,32.0,41.0,0.0,10010.0,10.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56022515,LOUIS,MEEKS,1.0,10.0,1.0,NaN,0.0,9999,410,0,NaN,NaN,NaN,NaN,NaN,
56022516,MARGARET,MEEKS,1.0,9.0,2.0,NaN,0.0,9999,410,0,NaN,NaN,NaN,NaN,NaN,
56022517,SUSAN,MEEKS,1.0,6.0,2.0,NaN,0.0,9999,410,0,NaN,NaN,NaN,NaN,NaN,
56022518,MOSES,MEEKS,1.0,0.0,1.0,NaN,0.0,9999,410,0,NaN,NaN,NaN,NaN,NaN,


In [11]:
US_Census1880_dta.dtypes # us1880a_street, namefrst, namelast (objects).  are all strings.

namefrst             object
namelast             object
sursim              float64
age                 float64
sex                 float64
occhisco            float64
ind95us             float64
enumdist              int16
countyus              int16
metrous               int16
us1880a_region      float64
us1880a_stateicp    float64
us1880a_city        float64
us1880a_stcounty    float64
us1880a_county      float64
us1880a_street       object
dtype: object

In [13]:
US_Census1880_dta.us1880a_street.value_counts()

us1880a_street
                                    41590823
                                     5882038
*                                     118378
* *                                    92906
EAST                                   27930
                                      ...   
81 SOUTH M STREET                          1
20 WASHINGTON STREET                       1
22 S                                       1
8 SAINT MICHAEL                            1
97 SAINT MICHAEL                           1
Name: count, Length: 637666, dtype: int64

In [16]:
# US_Census1880_dta.to_feather("../int_data/US_Census1880.feather")
!ls -lha ../int_data/US_Census1880.feather

-rw-r--r-- 1 xiet13 cluster-users 1.5G Aug 19 11:07 ../int_data/US_Census1880.feather


In [22]:
%time
US_Census1880_dta = pd.read_feather("../int_data/US_Census1880.feather").head(5)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.11 µs


In [17]:
# US_Census1880_dta.to_parquet("../int_data/US_Census1880.parquet")
!ls -lha ../int_data/US_Census1880.parquet

-rw-r--r-- 1 xiet13 cluster-users 603M Aug 19 11:08 ../int_data/US_Census1880.parquet


In [23]:
%time
US_Census1880_dta = pd.read_parquet("../int_data/US_Census1880.parquet").head(5)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.72 µs
